In [ ]:
pip install fastapi uvicorn tensorflow opencv-python


In [ ]:
from fastapi import FastAPI, File, UploadFile
import uvicorn
import numpy as np
import cv2
from tensorflow.keras.models import load_model
from io import BytesIO
from PIL import Image

# Initialize FastAPI app
app = FastAPI()

# Load the best performing model (change the path to your trained model)
model = load_model('nested_unet_weights.h5', custom_objects={'dice_coefficient': dice_coefficient})

# Preprocess image
def preprocess_image(image: Image.Image):
    image = np.array(image)
    image = cv2.resize(image, (256, 256))  # Resize to model input size
    image = image / 255.0  # Normalize
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

# Prediction route
@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    # Load the uploaded image
    image = Image.open(BytesIO(await file.read()))

    # Preprocess the image
    image = preprocess_image(image)

    # Predict using the model
    prediction = model.predict(image)

    # Convert prediction to binary mask
    mask = (prediction > 0.5).astype(np.uint8)
    mask = mask[0, :, :, 0]  # Remove batch dimension and channel dimension

    # Resize mask back to original image size (optional, depending on the original input size)
    mask = cv2.resize(mask, (image.shape[2], image.shape[1]))  # Assuming 256x256 input

    # Convert mask to a format that can be returned
    _, mask_png = cv2.imencode('.png', mask * 255)  # Convert binary mask to image format

    return {
        "mask": mask_png.tobytes()
    }

# Run FastAPI
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)


In [ ]:
uvicorn app:app --reload


In [ ]:
pip install streamlit requests


In [ ]:
import streamlit as st
import requests
import numpy as np
from PIL import Image
from io import BytesIO
import cv2

# FastAPI URL (adjust this URL based on where FastAPI is running)
FASTAPI_URL = "http://localhost:8000/predict/"

# Streamlit App UI
st.title("Brain MRI Metastasis Segmentation")

# Allow user to upload an MRI image
uploaded_file = st.file_uploader("Upload a Brain MRI image", type=["jpg", "jpeg", "png", "tiff"])

if uploaded_file is not None:
    # Display the uploaded image
    st.image(uploaded_file, caption="Uploaded MRI Image", use_column_width=True)

    # Convert the uploaded file to a PIL image
    image = Image.open(uploaded_file)

    # Button to trigger segmentation
    if st.button("Segment Metastasis"):
        # Send the image to the FastAPI backend for segmentation
        image_bytes = BytesIO()
        image.save(image_bytes, format="PNG")
        files = {'file': image_bytes.getvalue()}

        response = requests.post(FASTAPI_URL, files=files)

        if response.status_code == 200:
            # Get the mask from the response
            mask = np.array(Image.open(BytesIO(response.content)))

            # Display the mask
            st.image(mask, caption="Segmented Metastasis Mask", use_column_width=True)
        else:
            st.error("Error in processing the image. Please try again.")


In [ ]:
uvicorn app:app --reload


In [ ]:
streamlit run streamlit_app.py
